In [23]:
import sys
print(sys.executable)


/Users/bernatutuncu/anaconda3/envs/sahtepara_env/bin/python


In [24]:
import os, glob
import cv2
import numpy as np
from skimage.feature import hog

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import joblib

IMG_SIZE = (512, 512)

HOG_PARAMS = dict(
    orientations=9,
    pixels_per_cell=(16, 16),
    cells_per_block=(2, 2),
    block_norm="L2-Hys"
)

FAKE_DIR = "dataset/Fake"
REAL_DIR = "dataset/Real"



In [25]:
def extract_features(path, img_size=IMG_SIZE, hog_params=HOG_PARAMS):
    img = cv2.imread(path)
    if img is None:
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, img_size)

    feat = hog(img, **hog_params)
    return feat



In [26]:
def load_images(folder, label):
    X, y = [], []
    for p in glob.glob(os.path.join(folder, "*")):
        feat = extract_features(p)
        if feat is not None:
            X.append(feat)
            y.append(label)
    return X, y

X_fake, y_fake = load_images(FAKE_DIR, 0)
X_real, y_real = load_images(REAL_DIR, 1)

X = np.array(X_fake + X_real, dtype=np.float32)
y = np.array(y_fake + y_real, dtype=np.int32)

print("Fake:", len(X_fake))
print("Real:", len(X_real))
print("Toplam:", len(X))


Fake: 378
Real: 421
Toplam: 799


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(class_weight="balanced"))
])

param_grid = {
    "svc__kernel": ["rbf"],
    "svc__C": [1, 3, 10, 30],
    "svc__gamma": ["scale", 0.03, 0.1]
}

gs = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=1,      # mac’te daha stabil
    verbose=2
)

gs.fit(X_train, y_train)

best_model = gs.best_estimator_
print("En iyi CV accuracy:", gs.best_score_)
print("En iyi parametreler:", gs.best_params_)

pred = best_model.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ........svc__C=1, svc__gamma=scale, svc__kernel=rbf; total time=   3.0s
[CV] END ........svc__C=1, svc__gamma=scale, svc__kernel=rbf; total time=   2.9s
[CV] END ........svc__C=1, svc__gamma=scale, svc__kernel=rbf; total time=   2.8s
[CV] END ........svc__C=1, svc__gamma=scale, svc__kernel=rbf; total time=   2.8s
[CV] END ........svc__C=1, svc__gamma=scale, svc__kernel=rbf; total time=   2.9s
[CV] END .........svc__C=1, svc__gamma=0.03, svc__kernel=rbf; total time=   2.8s
[CV] END .........svc__C=1, svc__gamma=0.03, svc__kernel=rbf; total time=   2.8s
[CV] END .........svc__C=1, svc__gamma=0.03, svc__kernel=rbf; total time=   2.8s
[CV] END .........svc__C=1, svc__gamma=0.03, svc__kernel=rbf; total time=   2.8s
[CV] END .........svc__C=1, svc__gamma=0.03, svc__kernel=rbf; total time=   2.9s
[CV] END ..........svc__C=1, svc__gamma=0.1, svc__kernel=rbf; total time=   2.8s
[CV] END ..........svc__C=1, svc__gamma=0.1, svc

In [27]:
joblib.dump(
    {
        "model": best_model,
        "img_size": IMG_SIZE,
        "hog_params": HOG_PARAMS
    },
    "real_fake_model.pkl"
)
print("Kaydedildi: real_fake_model.pkl")



Kaydedildi: real_fake_model.pkl


In [28]:
def predict_image(image_path, model_path="real_fake_model.pkl"):
    pack = joblib.load(model_path)
    model = pack["model"]
    img_size = pack["img_size"]
    hog_params = pack["hog_params"]

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Dosya yok: {image_path}")

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Görüntü okunamadı: {image_path}")

    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, img_size)
    feat = hog(img, **hog_params)

    pred = model.predict([feat])[0]
    return "Real" if pred == 1 else "Fake"



In [29]:
import random

fake_samples = random.sample(glob.glob("dataset/Fake/*"), 10)
real_samples = random.sample(glob.glob("dataset/Real/*"), 10)

print("FAKE testleri:")
for p in fake_samples:
    print(p, "→", predict_image(p))

print("\nREAL testleri:")
for p in real_samples:
    print(p, "→", predict_image(p))


FAKE testleri:
dataset/Fake/IMG-20250717-WA0237_aug0.jpg → Fake
dataset/Fake/IMG-20250717-WA0219.jpg → Fake
dataset/Fake/IMG-20250717-WA0201_aug2.jpg → Fake
dataset/Fake/IMG-20250717-WA0176_aug0.jpg → Fake
dataset/Fake/IMG-20250717-WA0233.jpg → Fake
dataset/Fake/IMG-20250717-WA0227_aug0.jpg → Fake
dataset/Fake/IMG-20250717-WA0147_aug2.jpg → Real
dataset/Fake/IMG-20250717-WA0179_aug2.jpg → Fake
dataset/Fake/IMG-20250717-WA0238.jpg → Fake
dataset/Fake/IMG-20250717-WA0137_aug1.jpg → Fake

REAL testleri:
dataset/Real/IMG-20250717-WA0043_aug1.jpg → Real
dataset/Real/IMG-20250717-WA0014_aug0.jpg → Real
dataset/Real/IMG-20250717-WA0127_aug0.jpg → Real
dataset/Real/IMG-20250717-WA0047_aug2.jpg → Real
dataset/Real/IMG-20250717-WA0029_aug1.jpg → Real
dataset/Real/IMG-20250717-WA0110_aug0.jpg → Real
dataset/Real/IMG-20250717-WA0046.jpg → Real
dataset/Real/IMG-20250717-WA0109.jpg → Real
dataset/Real/IMG-20250717-WA0013_aug2.jpg → Real
dataset/Real/IMG-20250717-WA0034.jpg → Real


In [43]:
fake_test = glob.glob("dataset/Fake/*")[0]
real_test = glob.glob("dataset/Real/*")[9]

print(predict_image(fake_test))
print(predict_image(real_test))




Fake
Real


In [44]:
print(predict_image("/Users/bernatutuncu/Desktop/rupi2.webp"))



Fake


In [ ]:
;